In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [0]:
## Device configuration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
## Hyper-parameters

sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.003

In [0]:
## MNIST dataset

train_dataset = torchvision.datasets.MNIST(root = '../../data/',
                                           train = True,
                                           transform = transforms.ToTensor(),
                                           download = True)

test_dataset = torchvision.datasets.MNIST(root = '../../data/',
                                          train = False,
                                          transform = transforms.ToTensor())

In [0]:
## Data loader

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)

In [0]:
## Bidirectional recurrent neural network (many to one)

class BiRNN(nn.Module):
  
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(BiRNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                        batch_first = True, bidirectional = True)
    self.fc = nn.Linear(hidden_size * 2, num_classes)    ## 2 for bidirection
    
  def forward(self, x):
    ## Set initial states
    h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)    ## 2 for bidirection
    c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
    
    ## Forward propagate LSTM
    out, _ = self.lstm(x, (h0, c0))    ## out: tensor of shape (batch_size, seq_length, hidden_size * 2)
    
    ## Decode the hidden state of the last time step
    out = self.fc(out[:, -1, :])
    return out
  
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [0]:
## Loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate)

In [8]:
## Train the model

total_step = len(train_loader)

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(-1, sequence_length, input_size).to(device)
    labels = labels.to(device)
    
    ## Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    ## Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (i + 1) % 100 == 0:
      print ('Epoch [{} / {}], Step [{} / {}], Loss: {:.4f}'
             .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1 / 2], Step [100 / 600], Loss: 0.5402
Epoch [1 / 2], Step [200 / 600], Loss: 0.2862
Epoch [1 / 2], Step [300 / 600], Loss: 0.2744
Epoch [1 / 2], Step [400 / 600], Loss: 0.0969
Epoch [1 / 2], Step [500 / 600], Loss: 0.1543
Epoch [1 / 2], Step [600 / 600], Loss: 0.0665
Epoch [2 / 2], Step [100 / 600], Loss: 0.0743
Epoch [2 / 2], Step [200 / 600], Loss: 0.0778
Epoch [2 / 2], Step [300 / 600], Loss: 0.0362
Epoch [2 / 2], Step [400 / 600], Loss: 0.1241
Epoch [2 / 2], Step [500 / 600], Loss: 0.0439
Epoch [2 / 2], Step [600 / 600], Loss: 0.0962


In [9]:
## Test the model

with torch.no_grad():
  
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.reshape(-1, sequence_length, input_size).to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
  print('Test Accuracy of the model on the 10000 test images: {} %'
        .format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 97.79 %


In [0]:
## Save the model checkpoint

torch.save(model.state_dict(), 'model.ckpt')